## Tóm tắt thuật toán
1. Đầu vào: Dữ liệu X và số lượng cluster cần tìm K

2. Đầu ra: Các center M  và label vector cho từng điểm dữ liệu Y

3. Thuật toán
* Bước 1: Chọn K điểm bất kỳ làm các center ban đầu.
* Bước 2: Phân mỗi điểm dữ liệu vào cluster có center gần nó nhất.
          Nếu việc gán dữ liệu vào từng cluster ở bước 2 không thay đổi so với vòng lặp trước nó thì ta dừng thuật toán.
* Bước 3: Cập nhật center cho từng cluster bằng cách lấy trung bình cộng của tất các các điểm dữ liệu đã được gán vào cluster  đó sau bước 2.
* Bước 4Quay lại bước 2.
Chúng ta có thể đảm bảo rằng thuật toán sẽ dừng lại sau một số hữu hạn vòng lặp. Thật vậy, vì hàm mất mát là một số dương và sau mỗi bước 2 hoặc 3, giá trị của hàm mất mát bị giảm đi. Theo kiến thức về dãy số trong chương trình cấp 3: nếu một dãy số giảm và bị chặn dưới thì nó hội tụ! Hơn nữa, số lượng cách phân nhóm cho toàn bộ dữ liệu là hữu hạn nên đến một lúc nào đó, hàm mất mát sẽ không thể thay đổi, và chúng ta có thể dừng thuật toán tại đây

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML # dùng để hiển thị
%matplotlib inline

In [ ]:
from sklearn.datasets import make_blobs  #  dùng để tạo dữ liệu giả

# (x,y)
X,y=make_blobs(n_samples=150,         # tổng số điểm sample
               n_features=2,          # số lượng feature(số chiều) default:2
               centers=3,             # số lượng cluster
               cluster_std=0.5,       # độ lệch chuẩn giữa các cluster
               shuffle=True,          #  có trộn các sample với nhau không
               random_state=0)  

plt.scatter(X[:,0],X[:,1],c='black',marker='o',s=50)
plt.grid()
plt.show()

In [ ]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=3,            # số cluster
            init='random',           # vị trí center của cluster  default: 'k-means++'
            n_init=10,               # số lần chọn center của cluster default: '10'  trong số lần chọn  , sẽ chọn ra model có  SSE nhỏ nhất
            max_iter=300,            # Tiến hành chạy k-means nhiều nhất bao nhiêu lần default: '300'
            tol=1e-04,               # Khi tiến hành hội tụ các điểm, sai số cho phép là bao nhiêu, default: '1e-04'
            random_state=0)

y_km = km.fit_predict(X)

In [ ]:
plt.scatter(X[y_km==0,0],         # hiển thị y_km（cluster）=0
                    X[y_km==0,1],
                    s=50,
                    c='lightgreen',
                    marker='s',
                    label='cluster 1')
plt.scatter(X[y_km==1,0], # hiển thị y_km（cluster）=1
                    X[y_km==1,1],
                    s=50,
                    c='orange',
                    marker='o',
                    label='cluster 2')
plt.scatter(X[y_km==2,0],# hiển thị y_km（cluster）=2
                   X[y_km==2,1],
                    s=50,
                    c='lightblue',
                    marker='v',
                    label='cluster 3')
plt.scatter(km.cluster_centers_[:,0],   # km.cluster_centers_ điểm trung tâm
                    km.cluster_centers_[:,1],
                    s=250,
                    marker='*',
                    c='red',
                    label='centroids')
plt.legend()
plt.grid()
plt.show()

Có 1 lưu ý nhỏ là một model k-means được đánh giá là tốt khi giá trị SSE là nhỏ nhất. Với model trên ta có thể lấy giá trị SSE bằng cách sau:

In [ ]:
print ('Distortion: %.2f'% km.inertia_)

## Hạn chế

1. Chúng ta cần biết số lượng cluster cần clustering --> Elbow method
Để ý thấy rằng trong thuật toán nêu trên, chúng ta cần biết đại lượng K
 là số lượng clusters. Trong thực tế, nhiều trường hợp chúng ta không xác định được giá trị này. Có một số phương pháp giúp xác định số lượng clusters, tôi sẽ dành thời gian nói về chúng sau nếu có dịp. Bạn đọc có thể tham khảo Elbow method - Determining the number of clusters in a data set.

2. Nghiệm cuối cùng phụ thuộc vào các centers được khởi tạo ban đầu
Tùy vào các center ban đầu mà thuật toán có thể có tốc độ hội tụ rất chậm. Có một vài cách khắc phục đó là:

    * Chạy K-means clustering nhiều lần với các center ban đầu khác nhau rồi chọn cách có hàm mất mát cuối cùng đạt giá trị nhỏ nhất.

    * K-means++ -Improve initialization algorithm - wiki.

3. Các cluster cần có só lượng điểm gần bằng nhau

4. Các cluster cần có dạng hình tròn
Tức các cluster tuân theo phân phối chuẩn và ma trận hiệp phương sai là ma trận đường chéo có các điểm trên đường chéo giống nhau.

5. Khi một cluster nằm phía trong 1 cluster khác
Đây là ví dụ kinh điển về việc K-means clustering không thể phân cụm dữ liệu. Một cách tự nhiên, chúng ta sẽ phân ra thành 4 cụm: mắt trái, mắt phải, miệng, xunh quanh mặt. Nhưng vì mắt và miệng nằm trong khuôn mặt nên K-means clustering không thực hiện được:
